In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
from sklearn import preprocessing
import glob





In [ ]:
def get_movie_title(fld_path, temp):
    file_names = glob.glob(fld_path)
    
    path_title = {}
    for file in file_names:
        name = file.split('\\')[-1]
        title = name.rsplit(' ', 1)[0]
        # print(name)
        title_list.append(name)
        path_title[name] = title
    return path_title
title_list = []
path = './fin_movies_aspect_clusterd_result/*.csv'
path_title_dic = get_movie_title(fld_path = path, temp = True)
# print(path_title_dic)
print(len(path_title_dic))

my_file = open('all_already_aspect_extract_movies.txt', "r")
already_extracted = my_file.read()
already_extracted_list = already_extracted.split("\n")
print(len(already_extracted_list))
my_file.close()

In [209]:
# get twp modules' similarity score
aspect_sentiment = pd.read_csv('./all_movies_aspect_sentiment_score_0423.csv')
bert_sentiment = pd.read_csv('./bert_predict_sentiment_score_0502.csv')
f_weight = [[0.009208,0.007208,0.002208,0.008208,0.00577]]
#acting,diretion,music,story,effect
# features = ['original_title','keywords_trans','production_companies_trans','genres_trans','director_trans','cast_trans','release_year']
aspect_sentiment.drop('title', inplace=True, axis=1)
aspect_sentiment = np.array(aspect_sentiment.values.tolist())
# print(aspect_sentiment)
f_weight = preprocessing.normalize(f_weight)
print(f_weight)
# print(cosine_sim_matrix.shape)
for w_index in range(len(f_weight[0])):
    aspect_sentiment[w_index] = aspect_sentiment[w_index] * f_weight[0][w_index]
print(len(aspect_sentiment[0]))

c_aspect_sentiment = aspect_sentiment.copy()
c_bert_sentiment = bert_sentiment.copy()
c_bert_sentiment.drop('title', inplace=True, axis=1)


[[0.59156814 0.4630781  0.141853   0.52732312 0.37069376]]
5


In [211]:
# bert_sentiment
# c_bert_sentiment = np.array(c_bert_sentiment)
# c_aspect_sentiment
c_bert_sentiment = np.array(c_bert_sentiment, dtype=float)
idx = [i for i, arr in enumerate(c_bert_sentiment) if not np.isfinite(arr).all()]
idx
# np.any(np.isnan(c_bert_sentiment))
# np.any(np.isfinite(c_bert_sentiment))
# len(aspect_sentiment_sim)

[]

In [212]:
aspect_sentiment_sim = cosine_similarity(c_aspect_sentiment, c_aspect_sentiment)
bert_sentiment_sim = cosine_similarity(c_bert_sentiment, c_bert_sentiment)
# movies index mapping
mapping = pd.Series(bert_sentiment.index,index = bert_sentiment['title'])





In [213]:
aspect_sentiment_sim

array([[1.        , 0.58073403, 0.9804573 , ..., 0.98453681, 0.97389944,
        0.82092108],
       [0.58073403, 1.        , 0.71692205, ..., 0.67038364, 0.62939888,
        0.5863111 ],
       [0.9804573 , 0.71692205, 1.        , ..., 0.9942152 , 0.98340233,
        0.81462295],
       ...,
       [0.98453681, 0.67038364, 0.9942152 , ..., 1.        , 0.99103271,
        0.77416395],
       [0.97389944, 0.62939888, 0.98340233, ..., 0.99103271, 1.        ,
        0.77851855],
       [0.82092108, 0.5863111 , 0.81462295, ..., 0.77416395, 0.77851855,
        1.        ]])

In [214]:
metadata_sim

array([[1.        , 0.32914976, 0.31384547, ..., 0.2842502 , 0.12931439,
        0.12953967],
       [0.32914976, 1.        , 0.31439387, ..., 0.23680063, 0.26090061,
        0.12899119],
       [0.31384547, 0.31439387, 1.        , ..., 0.31413521, 0.12784122,
        0.12761598],
       ...,
       [0.2842502 , 0.23680063, 0.31413521, ..., 1.        , 0.12947516,
        0.12924987],
       [0.12931439, 0.26090061, 0.12784122, ..., 0.12947516, 1.        ,
        0.12931439],
       [0.12953967, 0.12899119, 0.12761598, ..., 0.12924987, 0.12931439,
        1.        ]])

In [215]:
print(aspect_sentiment_sim.shape)
print(metadata_sim.shape)

(856, 856)
(856, 856)


In [216]:
#recommender function to recommend movies based on metadata
def recommend_movies_based_on_metadata(movie_input, dataset, sim, index, is_meta = False):
    movie_index = index[movie_input]
    try:
        if len(movie_index)>1:
            movie_index = movie_index[0]
    except:
        pass
    # print(movie_index)
    #get similarity values with other movies
    similarity_score = list(enumerate(sim[movie_index]))
    # similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    # print(similarity_score)
    # Get the scores of the 15 most similar movies. Ignore the first movie.
    # similarity_score = similarity_score[1:15]
    # movie_indices = [i[0] for i in similarity_score]
    # print(type(similarity_score))
    # print(similarity_score[0])
    # print(similarity_score)
    return np.array(similarity_score)
    # r_movie = dataset['title'].iloc[movie_indices]
    # return r_movie

In [217]:
print(mapping)

title
10 Cloverfield Lane             0
10 Things I Hate About You      1
12 Angry Men                    2
12 Years a Slave                3
127 Hours                       4
                             ... 
Youth                         851
Zero Dark Thirty              852
Zodiac                        853
Zombieland                    854
Zootopia                      855
Length: 856, dtype: int64


In [ ]:
# m_sim = recommend_movies_based_on_metadata('Wonder Woman',aspect_sentiment,aspect_sentiment_sim,mapping)
# print([t[1] for t in m_sim])
# print(type(m_sim))
get_recomm = recommend_movies_based_on_metadata('Wonder Woman',aspect_sentiment,aspect_sentiment_sim,mapping)
print(get_recomm)
get_recomm = get_recomm.tolist()
print(get_recomm)

In [218]:
import numpy as np
meta_data = pd.read_csv('all_movies_meta_wr_0418.csv')
metadata_sim = np.load('colla_sim_matrix_0423.npy')
# print(metadata_sim)
# meta_title_index = pd.read_csv('./movies_title_index.csv',header=None, index_col=0, squeeze=True)
aspect_sentiment_sim


array([[1.        , 0.58073403, 0.9804573 , ..., 0.98453681, 0.97389944,
        0.82092108],
       [0.58073403, 1.        , 0.71692205, ..., 0.67038364, 0.62939888,
        0.5863111 ],
       [0.9804573 , 0.71692205, 1.        , ..., 0.9942152 , 0.98340233,
        0.81462295],
       ...,
       [0.98453681, 0.67038364, 0.9942152 , ..., 1.        , 0.99103271,
        0.77416395],
       [0.97389944, 0.62939888, 0.98340233, ..., 0.99103271, 1.        ,
        0.77851855],
       [0.82092108, 0.5863111 , 0.81462295, ..., 0.77416395, 0.77851855,
        1.        ]])

In [86]:
print(len(metadata_sim))

856


In [14]:
#check recommendation(only recommand with meta)
similarity_score = sorted(meta_sim, key=lambda x: x[1], reverse=True)
similarity_score = similarity_score[1:15]
movie_indices = [i[0] for i in similarity_score]
r_movie = aspect_sentiment['title'].iloc[movie_indices]
print(r_movie)
# r_movie = r_movie.tolist()
# precision = 0
# tp = 0
# fp = 0
# for r in r_movie:
#     if r in top_rank:
#         tp += 1
#     else:
#         fp += 1
# print(tp/(tp+fp))

9                    Batman v Superman_ Dawn of Justice
95                                        Suicide Squad
56                                   Kong_ Skull Island
128                                            Warcraft
23                                       Doctor Strange
30              Fantastic Beasts and Where to Find Them
45                           How to Train Your Dragon 2
124                                      Thor_ Ragnarok
66                                   Mad Max_ Fury Road
98                             The Amazing Spider-Man 2
78     Pirates of the Caribbean_ Dead Men Tell No Tales
108           The Hobbit_ The Battle of the Five Armies
135                          X-Men_ Days of Future Past
111                                      The Lego Movie
Name: title, dtype: object


In [62]:
# meta_data.to_csv('meta_title_meta_139.csv', index=False)
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
cosine_sim_matrix = np.load('colla_sim_matrix_0423.npy')



In [219]:
print(cosine_sim_matrix)

[[1.         0.32914976 0.31384547 ... 0.2842502  0.12931439 0.12953967]
 [0.32914976 1.         0.31439387 ... 0.23680063 0.26090061 0.12899119]
 [0.31384547 0.31439387 1.         ... 0.31413521 0.12784122 0.12761598]
 ...
 [0.2842502  0.23680063 0.31413521 ... 1.         0.12947516 0.12924987]
 [0.12931439 0.26090061 0.12784122 ... 0.12947516 1.         0.12931439]
 [0.12953967 0.12899119 0.12761598 ... 0.12924987 0.12931439 1.        ]]


In [187]:
import pickle
with open("movies_with_related_0424", "rb") as fp:   # Unpickling
    m_r = pickle.load(fp)
len(m_r)


856

In [178]:
m_r['tt0451279']

['Aquaman',
 'Man of Steel',
 'Batman v Superman: Dawn of Justice',
 'Wonder Woman 1984',
 'Justice League',
 "Zack Snyder's Justice League",
 'Doctor Strange',
 'Venom',
 'Deadpool 2',
 'Batman v Superman: Dawn of Justice (Ultimate Edition)',
 'Captain America: Civil War',
 'Iron Man Three']

In [188]:

index = 0
movies_with_related = m_r.copy()
for m in m_r.keys():
    movies_with_related[bert_sentiment.title.tolist()[index]+'@'+m] = movies_with_related.pop(m)
    index += 1
movies_with_related.keys()

dict_keys(['10 Cloverfield Lane@tt1179933', '10 Things I Hate About You@tt0147800', '12 Angry Men@tt0050083', '12 Years a Slave@tt2024544', '127 Hours@tt1542344', '1408@tt0450385', '1941@tt0078723', '2012@tt1190080', '21@tt0478087', '21 Jump Street@tt1232829', '22 Jump Street@tt2294449', '28 Days Later...@tt0289043', '28 Weeks Later@tt0463854', '3 Idiots@tt1187043', '30 Days of Night@tt0389722', '300@tt0416449', '3_10 to Yuma@tt0381849', '42@tt0453562', '50 First Dates@tt0343660', '54@tt0120577', '8 Mile@tt0298203', 'A Beautiful Mind@tt0268978', "A Bug's Life@tt0120623", 'A Night at the Roxbury@tt0120770', 'A Nightmare on Elm Street@tt0087800', 'A Walk to Remember@tt0281358', 'About Time@tt2194499', 'Abraham Lincoln_ Vampire Hunter@tt1611224', 'Across the Universe@tt0445922', 'Airlift@tt4387040', 'Alexander@tt0346491', 'Ali@tt0248667', 'Alice in Wonderland@tt1014759', 'Aliens@tt2316204', 'Alien_ Covenant@tt0118583', 'Alien_ Resurrection@tt0090605', "All the President's Men@tt0074119", 

In [189]:
len(movies_with_related)

856

In [95]:
movies_with_related['Beauty and the Beast']
# len(movies_with_related)

['The Little Mermaid',
 'Aladdin',
 'Mulan',
 'Cinderella',
 'Pocahontas',
 'Hercules',
 'Beauty and the Beast',
 'Tarzan',
 'Sleeping Beauty',
 'Snow White and the Seven Dwarfs',
 'The Hunchback of Notre Dame',
 'The Jungle Book']

In [232]:
#weighted-- meta and sentiment
w_range = np.linspace(0,1,11)

movies_precision = []
index = 0
for m, relate in movies_with_related.items():
    movie = m.split('@')[0]
    p_list = []
    for w1 in w_range:
        #sentitment module
        a_sim = recommend_movies_based_on_metadata(movie, bert_sentiment, aspect_sentiment_sim, mapping)
        b_sim = recommend_movies_based_on_metadata(movie, bert_sentiment, bert_sentiment_sim, mapping)
        sentiment_sim = a_sim * 0 + b_sim * 1
        
        # print(sentiment_sim)
        # print(w1,"sentiment_sim:",sentiment_sim)
        # aspect module
        meta_sim = recommend_movies_based_on_metadata(movie, meta_data, metadata_sim, mapping)
        mix_sim = sentiment_sim * w1 + meta_sim * (1 - w1)
        
        # print(w1,"mix_sim:",mix_sim)
        try:
            # print(len(mix_sim))
            similarity_score = sorted(mix_sim, key=lambda x: x[1], reverse=True)
        except:
            print(index)
            print(m)
            print(len(mix_sim))
            print(len(sentiment_sim))
            print(mix_sim)
            raise RuntimeError
        similarity_score = similarity_score[1:11]
        movie_indices = [i[0] for i in similarity_score]
        r_movie = bert_sentiment['title'].iloc[movie_indices]
        
        r_movie = r_movie.tolist()
        # print(r_movie)
        precision = 0
        tp = 0
        fp = 0
        for r in r_movie:
            if r in relate[0:10]:
                tp += 1
            else:
                fp += 1
        # print(tp,fp)
        p_list.append(tp/(tp+fp))
    # print(p_list)
    index+=1
    movies_precision.append(p_list)
    # print(max(p_list, key = p_list.get))



#0.0~0.3  





In [264]:
#weighted-- meta and sentiment

w_range = np.linspace(0,1,11)
p_list = {}
for w1 in w_range:
    #sentitment module
    title = 'I, Robot'
    imdb ='@tt0343818'
    # title = 'Good Will Hunting'
    # imdb ='@tt0119217'
    a_sim = recommend_movies_based_on_metadata(title, bert_sentiment, aspect_sentiment_sim, mapping)
    b_sim = recommend_movies_based_on_metadata(title, bert_sentiment, bert_sentiment_sim, mapping)
    sentiment_sim = a_sim * 0.1 + b_sim * 0.9
    # print(w1,"sentiment_sim:",sentiment_sim)
    # aspect module
    meta_sim = recommend_movies_based_on_metadata(title, meta_data, metadata_sim, mapping)
    mix_sim = sentiment_sim * w1 + meta_sim * (1 - w1)
    # print(w1,"mix_sim:",mix_sim)
    similarity_score = sorted(mix_sim, key=lambda x: x[1], reverse=True)
    similarity_score = similarity_score[1:11]
    movie_indices = [i[0] for i in similarity_score]
    r_movie = bert_sentiment['title'].iloc[movie_indices]
    
    r_movie = r_movie.tolist()
    print(r_movie)
#     precision = 0
    tp = 0
    fp = 0
    for r in r_movie[:10]:
        if r in movies_with_related[title+imdb]:
            tp += 1
        else:
            fp += 1
    # print(tp,fp)
    p_list[w1] = (tp/(tp+fp))
    # print(tp+fp)

print(p_list)
print(max(p_list, key = p_list.get))



#0.0~0.3  





['Dark City', 'Knowing', 'The Matrix', 'Independence Day', 'Terminator 3_ Rise of the Machines', 'Men in Black 3', 'Lucy', 'Hancock', 'Looper', 'The Terminator']
['Dark City', 'Knowing', 'The Matrix', 'Independence Day', 'Terminator 3_ Rise of the Machines', 'Men in Black 3', 'Lucy', 'Hancock', 'The Terminator', 'Looper']
['Dark City', 'Knowing', 'The Matrix', 'Independence Day', 'Terminator 3_ Rise of the Machines', 'Men in Black 3', 'Lucy', 'Hancock', 'The Terminator', 'The Matrix Reloaded']
['Dark City', 'Knowing', 'Independence Day', 'The Matrix', 'Terminator 3_ Rise of the Machines', 'Men in Black 3', 'Lucy', 'Hancock', 'The Matrix Reloaded', 'The Terminator']
['Knowing', 'Dark City', 'Independence Day', 'Men in Black 3', 'Terminator 3_ Rise of the Machines', 'The Matrix', 'Lucy', 'Hancock', 'The Matrix Reloaded', 'The Matrix Revolutions']
['Knowing', 'Dark City', 'Independence Day', 'Men in Black 3', 'Terminator 3_ Rise of the Machines', 'Lucy', 'Hancock', 'The Matrix Reloaded', 

In [205]:
with open("movies_with_related_0428", "wb") as fp:   #Pickling
    pickle.dump(movies_with_related, fp)

In [190]:
# movies_with_related['3_10 to Yuma@tt0381849']
movies_with_related_for_cola = movies_with_related.copy()
keys = []
for i in movies_with_related_for_cola.keys():
    keys.append(i.replace('_', ':'))
print(len(keys))
ori_keys = list(movies_with_related.keys())
for j in range(len(keys)):
    movies_with_related_for_cola[(keys[j].split('@')[0]) + '@' + (ori_keys[j].split('@')[1])] = movies_with_related_for_cola.pop((ori_keys[j].split('@')[0]) + '@' + (ori_keys[j].split('@')[1]))
# movies_with_related['Ant-Man@tt0478970']
# 'Ant-Man,tt0478970'.split(',')[0]
len(movies_with_related_for_cola)

856


856

In [231]:
final_weight = np.array(movies_precision)
final_weight = final_weight.mean(axis=0)
# final_weight = [ '%.2f' % elem for elem in final_weight ]
final_weight = [float(i) * 10 for i in final_weight]
print('precision@10 at different weight:', final_weight)
# final_weight.argmax(axis=0)
print('precision@10 at best weight:', max(final_weight))


precision@10 at different weight: [1.053738317757009, 1.0595794392523359, 1.0619158878504669, 1.0712616822429903, 1.082943925233644, 1.087616822429906, 1.0712616822429895, 1.0502336448598124, 0.9392523364485982, 0.7394859813084145, 0.23714953271028047]
precision@10 at best weight: 1.087616822429906


In [197]:
movies_precision[50]

0.2

In [268]:
final_weight = np.array(movies_precision)
final_weight = final_weight.mean()
final_weight

0.031191588785046732

In [202]:
# colla_filtering
mine_pivot = pd.read_csv('colla_filter.csv')
w_range = np.linspace(0,1,11)
movies_precision = []
index = 0
movies_precision = []
for m, relate in movies_with_related_for_cola.items():
    movie = m.split('@')[0]
    movie_watched = mine_pivot[movie]
    similarity_with_other_movies = mine_pivot.corrwith(movie_watched)
    similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending=False)
    r_movie = list(similarity_with_other_movies.to_dict().keys())[1:6]
    tp = 0
    fp = 0
    for r in r_movie:
        if r in relate[0:5]:
            tp += 1
        else:
            fp += 1
    movies_precision.append(tp/(tp+fp))



In [235]:
len(pure_sentiment_sim)

855

In [267]:
# pure muli-label sentiment compare model
pure_sentiment = pd.read_csv('./for_sentiment_compare_0501.csv')
pure_sentiment = np.array(pure_sentiment, dtype=float)
pure_sentiment_sim = cosine_similarity(pure_sentiment, pure_sentiment)
w_range = np.linspace(0,1,11)
movies_precision = []
index = 0
movies_precision = []
for m, relate in movies_with_related.items():
    movie = m.split('@')[0]
    pure_sim = recommend_movies_based_on_metadata(movie, bert_sentiment, pure_sentiment_sim, mapping)
    similarity_score = sorted(pure_sim, key=lambda x: x[1], reverse=True)
    similarity_score = similarity_score[1:11]
    movie_indices = [i[0] for i in similarity_score]
    r_movie = bert_sentiment['title'].iloc[movie_indices]
    r_movie = r_movie.tolist()
    # print(r_movie)
    precision = 0
    tp = 0
    fp = 0
    for r in r_movie:
        if r in relate[0:10]:
            tp += 1
        else:
            fp += 1
    movies_precision.append(tp/(tp+fp))